# 2.2 CTEs y Recursividad

**Resumen Ejecutivo**: Las Common Table Expressions (CTEs) estructuran consultas complejas en bloques legibles y reutilizables; las CTEs recursivas permiten procesar jerarquías y niveles agregados sin procedimientos iterativos.

---

## 🎯 Objetivos de Aprendizaje
- Refactorizar consultas complejas con CTEs para legibilidad y mantenibilidad.
- Combinar múltiples CTEs (ingresos, margen) y aplicar ranking/filtrado.
- Entender la sintaxis básica de CTEs recursivas para jerarquías.

## 🧰 Prerrequisitos
- Haber completado `01_fundamentos_window_functions.ipynb`.
- Conocer subconsultas y agregaciones avanzadas.
- Familiaridad con estructuras jerárquicas conceptuales (categorías, organigramas).

---

## 📚 Conceptos Clave
- **CTE**: subconsulta nombrada (`WITH nombre AS (...)`) que mejora legibilidad.
- **CTEs encadenadas**: varias CTEs que se referencian entre sí para modularizar lógica.
- **CTE recursiva**: CTE que se llama a sí misma para procesar niveles (ej. árbol de categorías).
- **Caso base + recursión**: patrón estándar para jerarquías (nivel 0, luego unir con hijos).

> ⚠️ CTEs no usados penalizan legibilidad; CTEs recursivas sin límite pueden causar loops infinitos (añadir `MAXRECURSION`).

In [ ]:
-- CTE para margen por producto y filtro posterior
WITH margen AS (
    SELECT producto_id, SUM(cantidad*(p.precio_unitario-p.costo_unitario)) AS margen_total
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY producto_id
)
SELECT * FROM margen WHERE margen_total > 500;

-- Ejemplo conceptual recursivo (estructura jerárquica hipotética de categorías)

In [ ]:
-- Si existiera dim_categorias (id, parent_id, nombre) podríamos: 
-- WITH rec AS (
--   SELECT id, parent_id, nombre, 0 AS nivel FROM dim_categorias WHERE parent_id IS NULL
--   UNION ALL
--   SELECT c.id, c.parent_id, c.nombre, r.nivel+1
--   FROM dim_categorias c
--   JOIN rec r ON c.parent_id = r.id
-- )
-- SELECT * FROM rec;

---

## 🧪 Ejercicios Guiados
- 🟢 Crear CTE que calcule ingresos por producto y luego filtre top 3 (usar `RANK()`).
- 🟠 Añadir otra CTE para margen y combinar con ingresos (JOIN interno por producto).
- 🔴 Simular jerarquía sin tabla recursiva creando niveles agregados (ej: segmentos → producto) y sumar unidades por nivel en CTE escalonadas.

In [ ]:
WITH ingresos AS (
    SELECT p.producto_id, p.nombre, SUM(fv.cantidad*p.precio_unitario) AS ingresos
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY p.producto_id, p.nombre
), ranked AS (
    SELECT *, RANK() OVER (ORDER BY ingresos DESC) AS rk FROM ingresos
)
SELECT producto_id, nombre, ingresos FROM ranked WHERE rk <= 3;

In [ ]:
WITH ingresos AS (
    SELECT p.producto_id, SUM(fv.cantidad*p.precio_unitario) AS ingresos
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY p.producto_id
), margen AS (
    SELECT p.producto_id, SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen
    FROM dbo.fact_ventas fv
    JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
    GROUP BY p.producto_id
)
SELECT i.producto_id, i.ingresos, m.margen
FROM ingresos i JOIN margen m ON i.producto_id = m.producto_id;

---

## ⚠️ Errores Comunes
- Repetir lógica en múltiples CTEs sin necesidad (refactorizar en una sola).
- Olvidar cláusula final (`SELECT` fuera del `WITH`) y generar error de sintaxis.
- Crear CTEs no usados (penaliza legibilidad sin beneficio).
- CTEs recursivas sin límite (`MAXRECURSION`) pueden causar loops infinitos.

---

## ✅ Conclusiones
- CTEs modularizan consultas complejas y mejoran legibilidad al nombrar bloques lógicos.
- CTEs encadenadas permiten construir métricas paso a paso (ingresos → ranking → top).
- CTEs recursivas procesan jerarquías sin procedimientos; útil para organigramas y árboles de categorías.

---

## 🚀 Aplicación Práctica
- Modularizar KPIs complejos (ingresos, margen, rentabilidad) en CTEs reutilizables.
- Calcular rankings y filtros múltiples sin repetir lógica.
- Procesar estructuras jerárquicas (categorías, territorios) sin bucles manuales.

---

## 💼 Perspectiva de Negocio
- CTEs mejoran mantenibilidad: cambios en lógica de negocio se reflejan en un solo lugar.
- Análisis por capas (ingresos → costos → margen) facilita validación por equipos de finanzas.
- Jerarquías recursivas permiten drill-down dinámico en reportes ejecutivos.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 2  
**Módulo**: 2.2 CTEs y Recursividad  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 Nota sobre IA: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](01_fundamentos_window_functions.ipynb) | [Siguiente ➡️](03_subconsultas_avanzadas.ipynb)
---
